In [1]:
from MaxCutProblem import MaxCutProblem

In [2]:
problem = MaxCutProblem()

In [ ]:
import itertools
import time
import params
from QAOA import QAOArunner
from datetime import date
import pandas as pd
from solver import Solver
import ray


@ray.remote
def parallell_runner(parameters, graph,name):
    qaoa = QAOArunner(graph, simulation=True, param_initialization=parameters[1],qaoa_variant=parameters[0], warm_start=parameters[2])
    qaoa.build_circuit()
    qaoa.run()
    solver = Solver(graph)
    bitstring, value = solver.solve()
    return {'param_initialization': parameters[1], 'qaoa_variant': parameters[0], 'warm_start' : parameters[2],
        'depth': params.depth, 'graph_size': len(graph.nodes), 'graph_name' : name,
        'time_elapsed': qaoa.time_elapsed, 'quantum_func_evals': qaoa.fev, 'obj_func_evolution': qaoa.objective_func_vals,
        'quantum_solution':qaoa.solution, 'quantum_obj_value' : qaoa.evaluate_sample(), 
        'classic_solution' : bitstring, 'classic_value': value }
if ray.is_initialized():
    ray.shutdown()
    print('Shutting down old Ray instance.')
ray.init(num_cpus=15)

iterables = [params.supported_qaoa_variants, params.supported_param_inits, [True]]
settings = list(itertools.product(*iterables))
print(settings)

settings = [[]]

data = []
# Reset results_underway.csv file
with open('results_underway.csv', 'w') as f:
    f.write('index,param_initialization,qaoa_variant,warm_start,depth,graph_size,graph_name,time_elapsed,quantum_func_evals,quantum_solution,quantum_obj_value,classic_solution,classic_value,obj_func_evolution,\n')

for parameters in settings:
    print('Parameters:', parameters)
    graphs = []
    names = []
    for i in range(5,10):
        graphs_i, names_i = problem.get_test_graphs(i)
        graphs.append(graphs_i) #TODO: check that this works for very small values
        names.append(names_i)
    graphs = list(itertools.chain.from_iterable(graphs))
    names = list(itertools.chain.from_iterable(names))
    futures = [parallell_runner.remote(parameters, graph, name) for graph, name in zip(graphs, names)]
    results = ray.get(futures)
    underway_df = pd.DataFrame(results)
    underway_df.to_csv(f'results_underway.csv', mode='a', header=False)
    data.extend(results)
    print(f'Done with Parameters: {parameters} at time: {time.time()}')
df = pd.DataFrame(data)
df.to_csv(f'results_.csv')

Shutting down old Ray instance.


2024-12-04 10:08:55,514	INFO worker.py:1810 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


[('vanilla', 'uniform', True), ('vanilla', 'gaussian', True), ('multiangle', 'uniform', True), ('multiangle', 'gaussian', True)]
Parameters: ('vanilla', 'uniform', True)
(parallell_runner pid=19776) You are running on the local simulator:  fake_brisbane
(parallell_runner pid=25540) {6: 511, 26: 104, 12: 109, 3: 272, 20: 675, 2: 536, 22: 155, 29: 569, 27: 83, 16: 726, 7: 244, 10: 524, 11: 654, 31: 105, 0: 116, 15: 701, 13: 225, 4: 90, 17: 547, 25: 525, 18: 218, 21: 536, 9: 171, 8: 90, 14: 572, 28: 264, 24: 254, 1: 83, 23: 76, 5: 84, 30: 75, 19: 106}
(parallell_runner pid=25540) {6: 0.0511, 26: 0.0104, 12: 0.0109, 3: 0.0272, 20: 0.0675, 2: 0.0536, 22: 0.0155, 29: 0.0569, 27: 0.0083, 16: 0.0726, 7: 0.0244, 10: 0.0524, 11: 0.0654, 31: 0.0105, 0: 0.0116, 15: 0.0701, 13: 0.0225, 4: 0.009, 17: 0.0547, 25: 0.0525, 18: 0.0218, 21: 0.0536, 9: 0.0171, 8: 0.009, 14: 0.0572, 28: 0.0264, 24: 0.0254, 1: 0.0083, 23: 0.0076, 5: 0.0084, 30: 0.0075, 19: 0.0106}
(parallell_runner pid=1828) You are running

RayTaskError(ValueError): [36mray::parallell_runner()[39m (pid=7400, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1862, in ray._raylet.execute_task
  File "C:\Users\torbjsme\AppData\Local\Temp\ipykernel_28600\245689880.py", line 15, in parallell_runner
  File "c:\Users\torbjsme\Master\QAOA\QAOA.py", line 224, in run
    init_params = self.get_init_params()
  File "c:\Users\torbjsme\Master\QAOA\QAOA.py", line 206, in get_init_params
    init_params = np.array([[np.random.normal(np.pi,0.2,param_cost_length), (np.random.normal(np.pi/2,0.2,param_mixer_length))] for i in range(params.depth)])
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (3, 2) + inhomogeneous part.

(parallell_runner pid=1828) You are running on the local simulator:  fake_brisbane
(parallell_runner pid=16256) You are running on the local simulator:  fake_brisbane
(parallell_runner pid=7400) You are running on the local simulator:  fake_brisbane [repeated 33x across cluster]
(parallell_runner pid=1828) You are running on the local simulator:  fake_brisbane [repeated 22x across cluster]
(parallell_runner pid=16256) You are running on the local simulator:  fake_brisbane [repeated 17x across cluster]
(parallell_runner pid=11284) {6: 94, 3: 2257, 21: 162, 24: 587, 22: 313, 1: 216, 28: 2301, 19: 122, 8: 159, 11: 239, 7: 582, 30: 224, 29: 278, 13: 196, 9: 316, 2: 288, 10: 156, 20: 215, 0: 87, 5: 88, 27: 43, 23: 168, 15: 112, 4: 34, 12: 100, 18: 192, 25: 107, 26: 59, 31: 85, 16: 112, 14: 49, 17: 59}
(parallell_runner pid=11284) {6: 0.0094, 3: 0.2257, 21: 0.0162, 24: 0.0587, 22: 0.0313, 1: 0.0216, 28: 0.2301, 19: 0.0122, 8: 0.0159, 11: 0.0239, 7: 0.0582, 30: 0.0224, 29: 0.0278, 13: 0.0196,

In [ ]:
results = ray.get(futures)
underway_df = pd.DataFrame(results)
underway_df.to_csv(f'results_underway.csv', mode='a', header=False)
data.extend(results)

In [ ]:
results2 = []
for task in futures:
    try:
        result = ray.get(task)  # Retrieve the result
        results2.append(result)  # Store in local memory
    except Exception as e:
        print(f"Task failed with error: {e}")